Get COVID data from Worldometer via Web Scraping

https://toarches.medium.com/mapping-coronavirus-b89880ecd400
    
    

In [1]:
#import libraries
import os, sys
import json
import pandas as pd
# import geopandas as gpd
import numpy as np
from numpy import int64
import requests, io
import urllib.request
# these are throwing an error when imported
#
# import folium
# from folium import plugins
# import fiona
# import branca
# from branca.colormap import linear
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import logging

logging.basicConfig(level=logging.DEBUG)

In [2]:
# url = 'https://www.worldometers.info/coronavirus/#countries'
url = 'https://www.worldometers.info/coronavirus/usa/california/'
response = requests.get(url)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.worldometers.info:443
DEBUG:urllib3.connectionpool:https://www.worldometers.info:443 "GET /coronavirus/usa/california/ HTTP/1.1" 200 None


In [3]:

data = response.content.decode('utf-8')


In order to clean the above messy data, we have to parse the content we get from the request. So, now we can define our HTML table parser object. I found there are some table parser functions available to get HTML table data. There is one table parser function I prefer to use as below:

In [4]:
class HTMLTableParser:

    def parse_url(self, url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')

        # num_tables = len(soup.find_all('table'))
        # logging.info(f'num_tables: {num_tables}')

        return [(table['id'], self.parse_html_table(table)) \
                for table in soup.find_all('table')]

    def parse_html_table(self, table):
        n_columns = 0
        n_rows = 0
        column_names = []

        # Find number of rows and columns
        # we also find the column titles if we can
        for row in table.find_all('tr'):

            # Determine the number of rows in the table
            td_tags = row.find_all('td')
            if len(td_tags) > 0:
                n_rows += 1
            if n_columns == 0:
                # Set the number of columns for our table
                n_columns = len(td_tags)

            # Handle column names if we find them
            th_tags = row.find_all('th')
            if len(th_tags) > 0 and len(column_names) == 0:
                for th in th_tags:
                    column_names.append(th.get_text())
                n_columns = len(column_names)

        # Safeguard on Column Titles
        logging.debug(f'n_rows: {n_rows}, n_columns: {n_columns}')
        # pp(column_names)
        if len(column_names) > 0 and len(column_names) != n_columns:
            raise Exception("Column titles do not match the number of columns")


        columns = column_names if len(column_names) > 0 else range(0, n_columns)
        df = pd.DataFrame(columns=columns,
                          index=range(0, n_rows))
        row_marker = 0

        for row in table.find_all('tr'):
            column_marker = 0
            columns = row.find_all('td')

            for column in columns:
                df.iat[row_marker, column_marker] = column.get_text()
                column_marker += 1

            if len(columns) > 0:
                row_marker += 1

        # Convert to float if possible
        for col in df:
            try:
                df[col] = df[col].astype(float)
            except ValueError:
                pass

        return df

In [5]:
hp = HTMLTableParser()
table_today = hp.parse_url(url)[0][1] # Grabbing the table from the tuple
table_today.head(10)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.worldometers.info:443
DEBUG:urllib3.connectionpool:https://www.worldometers.info:443 "GET /coronavirus/usa/california/ HTTP/1.1" 200 None
DEBUG:root:n_rows: 63, n_columns: 8
DEBUG:root:n_rows: 63, n_columns: 7


,County,TotalCases,NewCases,TotalDeaths,NewDeaths,ActiveCases,TotalTests,Source
0,\nCalifornia Total\n,"2,392,623",\n,"26,533",\n,"1,384,673","\n33,391,442",
1,\nLos Angeles,"806,210",\n,"\n10,682",,\nN/A,"\n4,699,140",\n[county] [state]
2,\nSan Bernardino,"203,651",\n,"\n1,445",,"\n29,521","\n1,666,893",\n[county] [state]
3,\nRiverside,"188,674",\n,"\n 1,985",,"\n67,853","\n1,653,692",\n[county] [state]
4,\nOrange,"165,440",\n,"\n1,901",,"\n71,860","\n2,076,496",\n[county] [state]
5,\nSan Diego,"164,500",\n,"\n1,592",,"\n41,528","\n2,780,950",\n[county] [state]
6,\nSanta Clara,"73,493",\n,\n747,,"\n72,746","\n1,941,588",\n[county] [state]
7,\nKern,"70,358",\n,\n512,,"\n49,376","\n343,467",\n[county] [state]
8,\nFresno,"67,484",\n,\n711,,\nN/A,"\n517,191",\n[county] [state]
9,\nSacramento,"66,956",\n,\n857,,"\n16,460","\n936,897",\n[county] [state]


We can see a few special characters (“\n”, “+”) to remove in the table. Let’s check the dataframe.


In [6]:
#check bottom rows
table_today.tail(10)

,County,TotalCases,NewCases,TotalDeaths,NewDeaths,ActiveCases,TotalTests,Source
53,\nPlumas,486,\n,\n4,,\n42,"\n6,941",\n[county] [state]
54,\nModoc,326,\n,\n1,,\n79,"\n6,947",\n[county]* [state]
55,\nTrinity,276,\n,\n4,,\n11,"\n5,237",\n[county] [state]
56,\nMariposa,265,\n,\n4,,\n47,"\n11,686",\n[county] [state]
57,\nAlpine,68,\n,\n,,\n3,\n,\n[county] [state]
58,\nSierra,47,\n,\n,,\n10,\n,\n[county] [state]
59,\n Alameda - Berkeley,,\n,\n,,\n,\n,\n[county] [state]
60,\nYuba-Sutter,,\n,\n,,\n,\n,\n[county] [state]
61,Unassigned,\n,\n,\n,\n,\n,"\n9,207,647",
62,Total:,"2,392,623",\n,"26,533",\n,"1,384,673","\n33,391,442",


There are some extra special characters (\n..\n) in the dataframe. We need to remove the extra characters. We only need country data for mapping in this tutorial. So we can drop the extra top and bottom rows that we do not need for data processing.

In [7]:
#Drop top buttom unwanted rows
df_today = table_today
# df_today = table_today.drop(table_today.index[[0]]).reset_index(drop=True)

#drop tail unwanted rows - last 2 (total row)
df_today.drop(df_today.tail(1).index,inplace=True)
#drop new line '\n' charachter 
df_today.replace(['\n'], '', regex=True, inplace=True)
df_today.replace([','], '', regex=True, inplace=True)

In [8]:
df_today.head(10)

,County,TotalCases,NewCases,TotalDeaths,NewDeaths,ActiveCases,TotalTests,Source
0,California Total,2392623,,26533,,1384673,33391442,
1,Los Angeles,806210,,10682,,N/A,4699140,[county] [state]
2,San Bernardino,203651,,1445,,29521,1666893,[county] [state]
3,Riverside,188674,,1985,,67853,1653692,[county] [state]
4,Orange,165440,,1901,,71860,2076496,[county] [state]
5,San Diego,164500,,1592,,41528,2780950,[county] [state]
6,Santa Clara,73493,,747,,72746,1941588,[county] [state]
7,Kern,70358,,512,,49376,343467,[county] [state]
8,Fresno,67484,,711,,N/A,517191,[county] [state]
9,Sacramento,66956,,857,,16460,936897,[county] [state]


In [9]:
df_today.tail(10)

,County,TotalCases,NewCases,TotalDeaths,NewDeaths,ActiveCases,TotalTests,Source
52,Inyo,635,,20,,125,13260,[county] [state]
53,Plumas,486,,4,,42,6941,[county] [state]
54,Modoc,326,,1,,79,6947,[county]* [state]
55,Trinity,276,,4,,11,5237,[county] [state]
56,Mariposa,265,,4,,47,11686,[county] [state]
57,Alpine,68,,,,3,,[county] [state]
58,Sierra,47,,,,10,,[county] [state]
59,Alameda - Berkeley,,,,,,,[county] [state]
60,Yuba-Sutter,,,,,,,[county] [state]
61,Unassigned,,,,,,9207647,


We need to format the table before starting mapping. The special characters in the dataframe can be removed using a loop as below:

In [10]:
for col in df_today.columns: 
    df_today[col]=df_today[col].str.replace("+", "").str.replace(",", "").str.replace("N/A", "").str.replace(" ", "").str.replace(" ", "")


All the extracted data is in text format and some column names are improper for data processing. We need to rename some column names.

In [11]:
# df1 = df.rename(columns={'Country,Other': 'COUNTRY_NAME', 'Serious,Critical': 'Serious_Critical' })
new_column_names = [col.lower().replace('\xa0','_').replace(' ', '_').replace('\n','').replace('/','_') for col in df_today.columns]
df_today.columns = new_column_names
df_today.columns

Index(['county', 'totalcases', 'newcases', 'totaldeaths', 'newdeaths',
       'activecases', 'totaltests', 'source'],
      dtype='object')

However, it is still not enough for data processing. We need to check the data type of each data frame column.


The data type of each column is object in the dataframe. So we need to convert some data types to appropriate data types in the data frame. Type conversion is the conversion of object from one data type to another data type.

For our purpose, we want there to be '' in columns if data is not updated yet

In [13]:
#convert object columns in dataframe to numeric
df_today.fillna('', inplace=True)
df_today.replace(np.nan, '', inplace=True)
df_today.replace(np.inf, '', inplace=True)
# for col in df1.columns[1:11]: 
#     df_today[col] = df_today.to_numeric(df1[col], errors='ignore')

It can be seen that web scraping is hard. We need to identify incomplete, incorrect, inaccurate, or irrelevant parts of the data and then replace, modify, or remove coarse data in the data frame.


In [32]:
# df_today.sort_values(by=['TotalCases'], inplace=True, ascending=False)
# df_today.head(10)

,#,COUNTRY_NAME,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,Serious_Critical,Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
0,1,USA,20904701,NaN,358682.0,NaN,12361387.0,NaN,8184632.0,29258.0,62968.0,1080,256748633,773370,331986837,North America,16,926,1
1,2,India,10324631,NaN,149471.0,NaN,9927310.0,783.0,247850.0,8944.0,7445.0,108,174899783,126112,1386864002,Asia,134,9278,8
2,3,Brazil,7716405,NaN,195742.0,NaN,6769420.0,NaN,751243.0,8318.0,36172.0,918,28600000,134068,213324870,South America,28,1090,7
3,4,Russia,3236787,24150.0,58506.0,504.0,2618882.0,19847.0,559399.0,2300.0,22175.0,401,91600000,627542,145966312,Europe,45,2495,2
4,5,France,2643239,NaN,64921.0,NaN,195174.0,NaN,2383144.0,2641.0,40449.0,993,35790511,547700,65346936,Europe,25,1007,2
5,6,UK,2599789,NaN,74570.0,NaN,NaN,NaN,NaN,1847.0,38195.0,1096,54892984,806456,68066909,Europe,26,913,1
6,7,Turkey,2232035,NaN,21295.0,NaN,2126432.0,NaN,84308.0,3764.0,26322.0,251,24811888,292601,84797583,Asia,38,3982,3
7,8,Italy,2141201,NaN,74985.0,NaN,1489154.0,NaN,577062.0,2569.0,35441.0,1241,26823305,443973,60416516,Europe,28,806,2
8,9,Spain,1936718,NaN,50837.0,NaN,NaN,NaN,NaN,2018.0,41415.0,1087,27016086,577711,46763996,Europe,24,920,2
9,10,Germany,1773540,NaN,34859.0,NaN,1368100.0,NaN,370581.0,5726.0,21134.0,415,34801593,414699,83920039,Europe,47,2407,2
